In [1]:
!pip install ReliefF
!pip install category_encoders

     |████████████████████████████████| 48 kB 1.7 MB/s 
  Created wheel for ReliefF: filename=ReliefF-0.1.2-py3-none-any.whl size=6318 sha256=addf8e1aa08e7d9d650b01aca37d012db5d71b1c27b708a539af55bb26b15516
  Stored in directory: /root/.cache/pip/wheels/18/8a/4a/3aada56cb869df4c1b1ed6af509c941a25985d1d1258a2e78c
Successfully built ReliefF
     |████████████████████████████████| 82 kB 345 kB/s 


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB

from ReliefF import ReliefF

import category_encoders as ce

from scipy.stats import chi2_contingency

np.random.seed(1)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
#https://www.kaggle.com/klmsathishkumar/predict-your-bmi-here
def ordinal_encoding(df,col,mapping):
    ordinal_encoder = ce.OrdinalEncoder(cols = [col],return_df = True,mapping = [{'col':col,'mapping':mapping}])
    df_final = ordinal_encoder.fit_transform(df)
    return df_final

# Import Data

In [5]:
data_heart = pd.read_csv('Datasets/heart/Heart.csv')
data_heart

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


##CHI2

In [6]:
chi_heart = data_heart
ct = pd.crosstab(data_heart.thal,data_heart.target, margins=True)
ct

target,0,1,All
thal,,,
0,1,1,2
1,12,6,18
2,36,130,166
3,89,28,117
All,138,165,303


In [7]:
obs = np.array([ct.iloc[0][0:5].values,
                  ct.iloc[1][0:5].values])
obs

array([[ 1,  1,  2],
       [12,  6, 18]])

In [8]:
#Si el 2do valor es menor que 0.05, se puede quedar la columna
chi2_contingency(obs)[0:3]

(0.21978021978021986, 0.8959325840172705, 2)

In [9]:
chi_heart = chi_heart.drop(columns = ['target'])
chi_heart = chi_heart.drop(columns = ['trestbps'])
chi_heart = chi_heart.drop(columns = ['chol'])
chi_heart = chi_heart.drop(columns = ['fbs'])
chi_heart = chi_heart.drop(columns = ['restecg'])
chi_heart = chi_heart.drop(columns = ['oldpeak'])
chi_heart = chi_heart.drop(columns = ['slope'])
chi_heart = chi_heart.drop(columns = ['thal'])
#chi_heart = chi_heart.drop(columns = ['age'])
#chi_heart = chi_heart.drop(columns = ['thalach'])
chi_heart

,age,sex,cp,thalach,exang,ca
0,63,1,3,150,0,0
1,37,1,2,187,0,0
2,41,0,1,172,0,0
3,56,1,1,178,0,0
4,57,0,0,163,1,0
...,...,...,...,...,...,...
298,57,0,0,123,1,0
299,45,1,3,132,0,0
300,68,1,0,141,0,2
301,57,1,0,115,1,1


##RELIEF

In [10]:


y = np.array(data_heart['target'])

data_heart =  np.array(data_heart.drop(columns = ['target']))


fs = ReliefF(n_neighbors=5, n_features_to_keep=6)

relief_heart = fs.fit_transform(data_heart, y)
print(relief_heart)

[[1. 1. 0. 0. 0. 3.]
 [2. 1. 0. 0. 0. 2.]
 [2. 0. 0. 0. 2. 1.]
 ...
 [3. 1. 0. 2. 1. 0.]
 [3. 1. 1. 1. 1. 0.]
 [2. 0. 0. 1. 1. 1.]]


In [11]:
relief_heart = pd.DataFrame(relief_heart)
relief_heart

,0,1,2,3,4,5
0,1.0,1.0,0.0,0.0,0.0,3.0
1,2.0,1.0,0.0,0.0,0.0,2.0
2,2.0,0.0,0.0,0.0,2.0,1.0
3,2.0,1.0,0.0,0.0,2.0,1.0
4,2.0,0.0,1.0,0.0,2.0,0.0
...,...,...,...,...,...,...
298,3.0,0.0,1.0,0.0,1.0,0.0
299,3.0,1.0,0.0,0.0,1.0,3.0
300,3.0,1.0,0.0,2.0,1.0,0.0
301,3.0,1.0,1.0,1.0,1.0,0.0


# Preprocess

In [12]:
scaler = StandardScaler()
relief_heart = scaler.fit_transform(relief_heart)
chi_heart = scaler.fit_transform(chi_heart)

#Split Data

In [13]:
X_train_relief, X_test_relief, y_train_relief, y_test_relief = train_test_split(relief_heart, y, test_size=0.33, random_state=0)
X_train_chi, X_test_chi, y_train_chi, y_test_chi = train_test_split(chi_heart, y, test_size=0.33, random_state=0)

# Models

In [14]:
discriminative = LogisticRegression(random_state = 0)
generative = GaussianNB()
    
discriminative.fit(X_train_relief, y_train_relief)
generative.fit(X_train_relief, y_train_relief)
    
dis_relief = discriminative.score(X_test_relief, y_test_relief)
gen_relief = generative.score(X_test_relief, y_test_relief)

In [15]:
discriminative = LogisticRegression(random_state = 0)
generative = GaussianNB()
    
discriminative.fit(X_train_chi, y_train_chi)
generative.fit(X_train_chi, y_train_chi)
    
dis_chi = discriminative.score(X_test_chi, y_test_chi)
gen_chi = generative.score(X_test_chi, y_test_chi)

In [16]:
print (f'Dis_relief: {dis_relief}\nDis_chi: {dis_chi}\nGen_relief: {gen_relief}\nGen_chi: {gen_chi}\n')

Dis_relief: 0.78
Dis_chi: 0.77
Gen_relief: 0.81
Gen_chi: 0.77

